<a href="https://colab.research.google.com/github/ruchithelamp/music/blob/main/Milestone_2_Recession_Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#mounting Google Drive to this Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Connecting to the shared drive folder (NOTE: You must have a copy of this in your My Drive for this to work!!)
team_folder = '/content/drive/My Drive/music_anthro_M2'

#imports and such
import os
import json
import pandas as pd
import glob


##This whole next chunk of code is for collecting and aggregating the data files for each year for musicbrainz and discogs
  #Still need to add genuis and acousticbrainz once these are added to the music_data_unzipped file!!
data_path = '/content/drive/My Drive/music_anthro_M2/music_data_unzipped'
years = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]

#Opening and aggregating all discogs json files by year
df_discogs_by_year = {}

for year in years:
    discogs_folder = os.path.join(data_path, year, 'discogs')
    json_files = glob.glob(os.path.join(discogs_folder, '*.json'))

    year_data = []
    for f in json_files:
        try:
            with open(f, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if isinstance(content, list):
                    year_data.extend(content)
                elif isinstance(content, dict):
                    year_data.append(content)
        except:
            pass

    if year_data:
        df_discogs_by_year[year] = pd.DataFrame(year_data)

#Opening and aggregating all musicbrainz json files by year
df_musicbrainz_by_year = {}

for year in years:
    mb_folder = os.path.join(data_path, year, 'musicbrainz')
    json_files = glob.glob(os.path.join(mb_folder, '*.json'))

    all_records = []

    for f in json_files:
        try:
            with open(f, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if isinstance(content, dict) and 'recordings' in content:
                    all_records.extend(content['recordings'])
                elif isinstance(content, list):
                    all_records.extend(content)
                elif isinstance(content, dict):
                    all_records.append(content)
        except:
            pass

    if all_records:
        df_musicbrainz_by_year[year] = pd.DataFrame(all_records)

Mounted at /content/drive
